<font color='red'> 
# Computational Statistics for Data Analysis


# <div class="alert alert-error"><strong><center><small>Statistics is the discipline of using data samples to support claims about populations.<small></center></strong> </div>

# Statistics is based on two main concepts: #

* A **population** is a collection of objects, items (“units”) about which information is sought.

* A **sample** is a part of the population that is observed.

# Index


### 1 Descriptive Statistics.
* 1.1 Getting data
* 1.2 Data preparation
* 1.3 Improving data as a pandas DataFrame
* 1.4 Data cleaning and preparation
 

### 2 Exploratory Data Analysis.
* 2.1 Summarizing the data: mean, variance, median, quantiles & percentiles
* 2.2 Histogram
* 2.3 Data distributions
    * 2.3.1 PMF
    * 2.3.2 CDF
* 2.4 Outliers
* 2.5 Measuring asymmetry
    * 2.5.1 Skewness
    * 2.5.2 Pearson's median skewness coefficient
* 2.6 Relative risk
* 2.7 A firts glimpse to Conditional Probability


### 3 Probabilities. 
* 3.1 Probability rules
* 3.2 Binomial distribution
* 3.3 Monte Carlo


<center><img src="images/taft_puck_eggdig.jpg">
</center><center><img src="images/eggs.png"></center>
<center><small>Headline from Chicago Tribune June 13, 1897.</small></center>

### "Do first babies arrive late?"

***From [Think Stats: Probability and Statistics for Programmers](http://www.greenteapress.com/thinkstats/), by Allen B. Downey, published by O'Reilly Media.***

Some people believe it is true, but **without data analysis** to support it, this claim is a case of *anecdotal evidence*:

* There are a *small number of samples* (personal experience, friends, etc.).
* There is a *selection bias*: most *believers* are interested in this claim because their first babies were late.
* There is a *confirmation bias*: believers might be more likely to contribute data that confirm it.
* Sources are *innaccurate*: personal stories are subject to memory deformations. 



<font color='blue'>
    
# 1 Descriptive Statistics.
* 1.1 Getting data
* 1.2 Data preparation
* 1.3 Improving data as a pandas DataFrame
* 1.4 Data cleaning 
 <font\>

## 1.1 Getting Data

There is an interesting and publicly available **data source** to check the claim of babies arrival. Since 1973 the U.S. Centers for Disease Control and Prevention (CDC) have conducted a survey, the National Survey of Family Growth (NSFG), to gather *information on family life, marriage and divorce, pregnancy, infertility, use of contraception, and men's and women's health.* 

Data can be downloaded from: 

http://www.cdc.gov/nchs/nsfg/nsfg_cycle6.htm#cyc6downdatafiles

We will use this file: 

* Female Pregnancy Data File (2002FemPreg.dat): one record for each pregnancy reported by a respondent.

There are 13593 pregnancies in our data. The meaning of the data is (each record is in a line):

+ <code>case.id</code> is the ID of the respondent. From col 1 to 12.
+ <code>prg.lenght</code> is the duration of the pregnancy in weeks. From col 275 to 276. 
+ <code>outcome</code> is the outcome of the pregnancy (1 = live birth). Col 277.
+ <code>birth.ord</code> is the integer birth order of each live birth. From col 278 to 279.


<small> If curious: Online documentation of the survey is at http://www.icpsr.umich.edu/nsfg6.<small> 

### 1.2 Data preparation

One of the reasons we are using a general-purpose language such as Python rather than a stats language like R is that for many projects the *hard* part is preparing the data, not doing the analysis.

The most common steps are:

1. **Getting the data**. Data can be directly read from a file or it might be necessary to scrap the web.
2. **Parsing the data**.  Of course, this depends on what format it is in: plain text, fixed columns, CSV, XML, HTML, etc.
3. **Cleaning the data**.  Survey responses and other data files are almost always incomplete.  Sometimes there are multiple codes for things like, *not asked*, *did not know*, and *declined to answer*. And there are almost always errors. A simple strategy is to remove or ignore incomplete records.
4. **Building data structures**. Once you read the data, you usually want to store it in a data structure that lends itself to the analysis you want to do.

If the data fits into memory, building a data structure is usually the way to go.   If not, you could build a **database**, which is an out-of-memory data structure. Most databases provide a mapping from keys to values, so they are like dictionaries.

In [ ]:
file = open('files/2002FemPreg.dat', 'r')

# Let's build a list of lists.

preg=[]
for line in file:
    preg.append([int(line[:12]), int(line[274:276]), int(line[276]),
                 int(line[277:279])])

Ooops! There is something wrong in the data file!

By inspecting the data we can observe that there are some empty records that caused an error to the ``int`` function.

In [ ]:
file = open('files/2002FemPreg.dat', 'r')

def chr_int(a):
    if a == '  ':
        return 0
    else:
        return int(a)
        
preg=[]
for line in file:
    lst  = [int(line[:12]), int(line[274:276]), int(line[276]),
                 chr_int(line[277:279])]
    preg.append(lst)

In [ ]:
print('First record: ', preg[0])
print('The number of entries is: ', len(preg))

### 1.3 Importing data as a pandas DataFrame

In [ ]:
%matplotlib inline

import pandas as pd
df = pd.DataFrame(preg) #  Two-dimensional size-mutable, potentially heterogeneous tabular 
        #data structure with labeled axes 
    
df.columns = ['caseId', 'prgLength', 'outcome', 'birthOrd']
df.head()

In [ ]:
counts = df.groupby('birthOrd').size()
print('The number of first babies is: ',counts[1])
print('Number of babies according to their order: ',counts) 

# also: df.outcome.value_counts()

Let's count the number of  births according to the order.

Let's build a partition of *live births* into two groups: first babies and others.

In [ ]:
# Divide records into two lists: first babies and others.

firstbirth = df[ (df.outcome == 1) & (df.birthOrd == 1)]
print('The number of first babies is: ', firstbirth.shape[0])

In [ ]:
othersbirth = df[(df.outcome == 1) & (df.birthOrd >= 2)]
print('The number of others babies is: ', othersbirth.shape[0])

### 1.4 Data Cleaning

The most common steps are:

+ **Sample the data**. If the amount of raw data is huge, processing all of them may require an extensive amount of processing power which may not be practical.  


+ **Impute missing data**. It is quite common that some of the input records are incomplete in the sense that certain fields are missing or have input error.  Validate the format and value! 

In case of fields missing: 
<small>
* (a) Discard the whole record if it is incomplete; 
* (b) Infer the missing value based on the data from other records (fill the missing data with the average or the median).
<small>



+ **Normalize numeric value**. Normalize data is about transforming numeric data into a uniform range.
+ **Reduce dimensionality**. High dimensionality can be a problem for some machine learning methods.  There are two ways to reduce the number of input features.  One is about *removing* *irrelevant* input variables, another one is about $removing$ $redundant$ input variables.
+ **Add derived features**. In some cases, we may need to compute additional attributes from existing attributes (f.e. converting a geo-location to a zip code, or converting the age to an age group).
+ **Discretize numeric value into categories**. Discretize data is about cutting a continuous value into ranges and assigning the numeric with the corresponding bucket of the range it falls on.  For numeric attribute, a common way to generalize it is to discretize it into ranges, which can be either constant width (variable height/frequency) or variable width (constant height).
+ **Binarize categorical attributes**. Certain machine learning models only take binary input (or numeric input).  In this case, we need to convert categorical attribute into multiple binary attributes, while each binary attribute corresponds to a particular value of the category. 

+ **Select, combine, aggregate data**. Designing the form of training data is the most important part of the whole predictive modeling exercise because the accuracy largely depends on whether the input features are structured in an appropriate form that provide strong signals to the learning algorithm. Rather than using the raw data as it is, it is quite common that multiple pieces of raw data need to be combined together, or aggregating multiple raw data records along some dimensions (e.g. clinical vs. demographic).

<font color='blue'> 
      
## 2 Exploratory Data Analysis.
* 2.1 Summariizing the data: mean, variance, median, quantiles & percentiles
* 2.2 Histogram
* 2.3 Data distributions
* 2.3.1 PMF
* 2.3.2 CDF
* 2.4 Outliers
* 2.5 Measuring asymmetry
* 2.5.1 Skewness
* 2.5.2 Pearson's median skewness coefficient
* 2.6 Relative risk
* 2.7 A firts glimpse to Conditional Probability
<font\>

### 2.1 Summarizing the data: 
#### 2.1.1 Sample Mean 

If you have a sample of $n$ values, $x_i$, the **sample mean** is the sum of the values divided by the number of values:

$$ \mu = \frac{1}{n} \sum_i x_i$$

The **mean** is the most basic and important summary statistic. It describes the *central tendency* of a sample. 

Let's see if there is a difference between firstbirth and othersbirth:

In [ ]:
print('The mean of the first birth is: ', firstbirth['prgLength'].mean())
print('The mean of the non first birth is: ', othersbirth['prgLength'].mean())

In [ ]:
print('The difference in time is: ', abs(firstbirth['prgLength'].mean()-
          othersbirth['prgLength'].mean()),"weeks")

In [ ]:
print('The difference in time is: ',abs(firstbirth['prgLength'].mean()-
          othersbirth['prgLength'].mean())*7,"days")

In [ ]:
print('The difference in time is: ',abs(firstbirth['prgLength'].mean()-
          othersbirth['prgLength'].mean())*7*24, "hours")

This difference in sample means can be considered a first evidence of our hypothesis!


**Comment: ** Do not confuse both concepts: the sample mean and the population mean.  The first one is the mean of samples taken from the population and the second one is the mean of the whole population.

#### 2.1.2 Sample Variance

Usually, the mean is not a sufficient descriptor of the data, we can do a little better with two numbers: mean and **variance**:

$$ \sigma^2 = \frac{1}{n} \sum_i (x_i - \mu)^2 $$

**Variance** $\sigma^2$ describes the *spread* of data. The term $(x_i - \mu)$ is called the *deviation from the mean*, so variance is the mean squared deviation.

The square root of variance, $\sigma$, is called the **standard deviation**. We define standard deviation because variance is hard to interpret (in the case the units are grams, the variance is in grams squared).


In [ ]:
mu1 = firstbirth['prgLength'].mean()
mu2 = othersbirth['prgLength'].mean()
var1 = firstbirth['prgLength'].var()
var2 = othersbirth['prgLength'].var()
std1 = firstbirth['prgLength'].std()
std2 = othersbirth['prgLength'].std()
print('Firstbirth mu1:', mu1, 'var1:', var1, 'std1:', std1)
print('Othersbirhts mu2:', mu2, 'var2:', var2, 'std2:', std2)

#### 2.1.3 Sample Median

The statistical median is an order statistic that gives the *middle* value of a sample. It is a value more robust to ouliers.

In [ ]:
firstbirthmedian= firstbirth['prgLength'].median()
othersbirthmedian= othersbirth['prgLength'].median()
print('The median measures of both first- and otherbirths are: ', 
      firstbirthmedian, othersbirthmedian)

<center><img src="images/mean_median.gif">

#### 2.1.4 Summarizing the data: Quantiles & Percentiles

Order the sample $\{ x_i \}$, then find $x_p$ so that it divides the data into two parts where:

+ a fraction $p$ of the data values are less than or equal to $x_p$ and
+ the remaining fraction $(1 − p)$ are greater than $x_p$.

That value $x_p$ is the pth-quantile, or 100×pth percentile.

**5-number summary**: $x_{min}, Q_1, Q_2, Q_3, x_{max}$, where $Q_1$ is the 25×pth percentile,
$Q_2$ is the 50×pth percentile and $Q_3$ is the 75×pth percentile.

### 2.2 Histogram

The most common representation of a distribution is a [**histogram**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html), which is a graph that shows the frequency of each value.

In [ ]:
fb=firstbirth['prgLength']

fb.hist(bins=30)

In [ ]:
        
ob=othersbirth['prgLength']
ob.hist(density=0, bins=30)

In [ ]:
import seaborn as sns
#import warnings
#warnings.filterwarnings('ignore')

fb.hist(density=True, bins=30, alpha=.5)   # default number of bins = 10
ob.hist(density=True, bins=30, alpha=.5, color=sns.desaturate("indianred", .75))

In [ ]:
import scipy.stats as stats

# Computes several descriptive statistics:
# size of the data 
# minimum and maximum value of data array
# arithmetic mean 
# unbiased variance 
# biased skewness 
# kurtosis (Fisher)

print('Firstbirhts: ', stats.describe(fb.values))

print('Othersbirths: ', stats.describe(ob.values))

## 2.3 Data Distributions

Summarizing can be dangerous: very different data can be described by the same statistics. It must be validated by inspecting the data.

We can look at the **data distribution**, which describes how often (frequency) each value appears.


We can normalize the frequencies of the histogram by dividing/normalizing by $n$, the number of samples. The normalized histogram is called **Probability Mass Function (PMF)**.

In [ ]:
# if needed, execute the command 'pip3 install seaborn'

import seaborn as sns

fb.hist(density=True, alpha=.5)
ob.hist(density=True, alpha=.5, color=sns.desaturate("indianred", .75))

The **cumulative distribution function (CDF)**, or just distribution function, describes the probability that a real-valued random variable X with a given probability distribution will be found to have a value less than or equal to x. 

In [ ]:
fb.hist(density=1, cumulative=True, linewidth=3.5)

In [ ]:
ob.hist(density=1,  cumulative=True, linewidth=3.5)

In [ ]:
fb.hist(bins=10, density=1,  alpha=.5)   # default number of bins = 10
ob.hist(bins=10, density=1,  alpha=.5, color=sns.desaturate("indianred", .75))

# Check with 20, 30, 60 bins.

In [ ]:
fb.hist(density=True, histtype='step', cumulative=True,  linewidth=3.5, bins=30)
ob.hist(density=True, histtype='step', cumulative=True,  linewidth=3.5, bins=30, color=sns.desaturate("indianred", .75))

## 2.4 Outliers

**Outliers** are data samples with a value that is far from the central tendency.

We can find outliers by:

+ Computing samples that are *far* from the median.
+ Computing samples whose value *exceeds the mean* by 2 or 3 standard deviations.

In [ ]:
df[(df.outcome == 1) & (df['prgLength'] < df['prgLength'].median() - 10)]

In [ ]:
df[(df.outcome == 1) & (df['prgLength'] > df['prgLength'].median() + 6)]

If we think that outliers correspond to errors, an option is to trim the data by discarting the highest and lowest values.

In [ ]:
df2 = df.drop(df.index[(df.outcome == 1) & 
                       (df['prgLength'] > df['prgLength'].median() + 6)])

df3 = df2.drop(df2.index[(df2.outcome == 1) & 
                         (df2['prgLength'] < df2['prgLength'].median() - 10)])

In [ ]:
fb3 = df3[(df3.outcome == 1) & (df3.birthOrd == 1)]

mu3fb = fb3['prgLength'].mean()
std3fb = fb3['prgLength'].std()
md3fb = fb3['prgLength'].median()

print('Before outliers removing: ', mu1, std1, firstbirthmedian, 
          firstbirth['prgLength'].min(), firstbirth['prgLength'].max())
print('After outliers removing: ', mu3fb, std3fb, md3fb, 
          fb3['prgLength'].min(),fb3['prgLength'].max())

In [ ]:
ob3 = df3[(df3.outcome == 1) & (df3.birthOrd >= 2)]

mu3ob = ob3['prgLength'].mean()
std3ob = ob3['prgLength'].std()
md3ob = ob3['prgLength'].median()

print('Before outliers removing: ', mu2, std2, othersbirthmedian, 
          othersbirth['prgLength'].min(),othersbirth['prgLength'].max())
print('After outliers removing: ', mu3ob, std3ob, md3ob, 
          ob3['prgLength'].min(), ob3['prgLength'].max())

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,3))

df3.prgLength[(df3.outcome == 1)].plot(alpha=.5, color='blue')
df.prgLength[(df.outcome == 1)].plot(alpha=.5, 
                            color=sns.desaturate("indianred", .95))


Let's see what is happening near the mode:

In [ ]:
import numpy as np

countfb,divisionfb = np.histogram(fb3['prgLength']) #what is the difference with hist()?!
countob,divisionob = np.histogram(ob3['prgLength'])
print (countfb-countob)


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,3))
val = [(divisionfb[i]+divisionfb[i+1])/2 for i in range(len(divisionfb)-1)]
plt.plot(val, countfb-countob, 'o-', label='difference') 
plt.plot(val, countfb, 'r*-', label='First babies') 
plt.plot(val, countob, 'g+-', label='Other babies') 
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,3))
val = [(divisionfb[i]+divisionfb[i+1])/2 for i in range(len(divisionfb)-1)]
plt.plot(val, countfb-countob, 'o-', label='difference') 
plt.legend()

## There is still some evidence for our hypothesis!

### 2.5 Measuring asymmetry.

**Skewness** is a statistic that measures the asymmetry of set of $n$ data samples $x_i$:

$$ g_1 = \frac{m_3}{s^3}= \frac{\frac{1}{n} \sum_i (x_i - \mu)^3 }{[\frac{1}{n-1} \sum_i (x_i - \mu)^2]^{\frac{3}{2}} } $$

The  the numerator is the mean cubed deviation and the denominator  is the mean squared deviation (or variance) to power 3.

Negative deviation indicates that the distribution "skews left" (it extends farther to the left than to the right).

**Skewness** can be affected by outliers!!! A simpler alternative is to look at the relationship between mean ($\mu$) and median ($\mu_{\frac{1}{2}}$). 

**2.6 Pearson's median skewness coefficient** is a more robust alternative:

$$ g_p = \frac{3(\mu - \mu_{\frac{1}{2}})}{\sigma} $$

In [ ]:
fb33=fb3['prgLength']
ob33=ob3['prgLength']

print('Firstbirhts: ', stats.describe(fb33.values),'\n')
print('Otherbirhts: ', stats.describe(ob33.values))

**Exercise**: Write a function to compute $g_1$ and $g_p$ of the pregnancy length.

**Exercise**:

+ Read the file ``run10.txt`` from the ``files`` directory. It represents 16.924 runners who finished the 2012 Cherry Blossom 10 mile run in USA. The file is a ``tab``separated file. It can be read with the pandas ``read_table`` function.
+ Compute the mean time.
+ Compute the difference in mean between men and women.
+ Visualize both distributions (normalized histogram).

In [ ]:
## Your solution here

**Exercises**: 

+ Could you give a real example, where for all data samples, $x_i \leq \mu$? 
+ Could you give a real example, where for all data samples, $x_i \leq \mu_{\frac{1}{2}}$? This is really a distribution that skews left!
+ If we ask to a random group of people "What is your position with respect to the average driver?", what kind of distribution would we get? 

## 2.6 Relative Risk

Let's say that a baby is "early" if it is born during week 37 or earlier, "on time" if it is born during week 38, 39 or 40, and "late" if it is born during week 41 or later. 

**Let's compute the probability of being *early*, *on time* and *late* for first babies and the others.**

In [ ]:
print("Firsts babies: ")
print('Early: ',len(fb3[fb3['prgLength'] <38])/float(len(fb3.index)))

print('Late: ', len(fb3[fb3['prgLength'] >40])/float(len(fb3.index)))

print('On time: ',len(fb3[(fb3['prgLength'] >37)&(fb3['prgLength'] < 41)])/float(len(fb3.index)))

In [ ]:
print("Other babies:")
print("Early", len(ob3[ob3['prgLength'] <38])/float(len(ob3.index)))

print("Late", len(ob3[ob3['prgLength'] >40])/float(len(ob3.index)))

print("On time", len(ob3[(ob3['prgLength'] >37) &
                        (ob3['prgLength']<41)])/float(len(ob3.index)))

The **relative risk** is the ratio of two probabilities. In our case, the probability that a first baby is born early is 17%. For other babies is 16%, so the relative risk is:

In [ ]:
earlyfb = len(fb3[fb3['prgLength'] <38])/float(len(fb3.index))
earlyob = len(ob3[ob3['prgLength'] <38])/float(len(ob3.index))
print("First babies are about", earlyfb/earlyob, "more likely to be early.")

That means that first babies are about 7% more likely to be early. For the case of late births:

In [ ]:
latefb = len(fb3[fb3['prgLength'] >40])/float(len(fb3))
lateob = len(ob3[ob3['prgLength'] >40])/float(len(ob3))
print("First babies are about", latefb/lateob, "more likely to be late.")

That means that first babies are about 67% more likely to be late. 

## 2.7 A firts glimpse to Conditional Probability

Imagine that someone you know is pregnant and it is the beginning of week 39. What is the chance that the baby will be born in the week 39? What is the chance if it is a first baby?

We can ask these questions by computing a **conditional probability**, $P(X|Y)$.

In our first question, the event X is a birth in week 39 and the event Y is that we know that the baby didn't arrive during weeks 0-38. In the second question, we also know that it is a first baby.

A simple way to compute these chances is to drop from our data the cases that do not fulfill the conditions and then renormalize.

In [ ]:
df4 = df3.drop(df3.index[df3['prgLength'] < 39]) 

We are ready to compute the probability that the baby will be born in the week 39 for a pregnant woman in the beginning of week 39.

In [ ]:
print(len(df4[(df4.prgLength == 39)].index)/float(len(df4)))

Let's now add the second condition.

In [ ]:
fb39 = df4[(df4.birthOrd == 1)]
ob39 = df4[(df4.birthOrd > 1)]

fb39['prgLength'].hist(bins=6,  density=True, alpha=.5)   # default number of bins = 10, blue
ob39['prgLength'].hist(bins=6,  density=True, alpha=.5, color=sns.desaturate("indianred", .75))

In [ ]:
print('Probability First baby to be born on week 39:', 
    len(fb39[(fb39.prgLength == 39)].index)/
    float(len(fb39.index)))

In [ ]:
print('Probability non first baby to be born on week 39:',
    len(ob39[(ob39.prgLength == 39)].index)/
    float(len(ob39.index)))


### Discussions.

After exploring the data we have seen some **appearent effects** that seem to support our first hypothesis:

+ **Data description**: The mean pregnant lenght for first babies is 38.76 and for other babies is 38.65.

+ **Relative risk**: First babies are about 67% more likely to be late.

+ **Conditional probability**: If someone is pregnant and it is the beginning of week 39, the chance (63% vs. 72%) that the baby will be born in the week 39 is lower if it is the first baby.


#### Exercises: Other possible experiments

We can compare the first and others for the same woman. While may be unlikely it could still be that a tendency exists for a woman's second, third, etc, child comes earlier.

<small>(Result:  The second baby is born about some hours earlier, but this difference is not *statistically significant*.)<small>


## 3 Probabilities. 
* 3.1 Probability rules
* 3.2 Binomial distribution
* 3.3 Monte Carlo

The most common definition of **probability** is a *frequency expressed as a fraction* of the universe of possible outcomes. 

   -Thus, it is a real value between 0 and 1 that is intended to be a measure corresponding to the idea that some things are more likely than others.

**Frequentism** because it defines probability in terms of frequencies.

An alternative is **Bayesianism**, which defines probability as a degree of belief that an event will occur.

   -Example: What is the probability that Thaksin Shinawatra is the Prime Minister of Thailand?

The *things* we assign probabilities are called **events**, E.  A *situation* where E might or might not happen is called a **trial**.

   -In the case of a six-sided die, each roll is called a **trial**. If we want to compute P(6), each time a 6 appears is called a **success**. Other trials are called **failures**. 

If in a *finite series of n identical trials* we observe s successes, the **probability of the success** is s/n.



### 3.1 Probability Rules 


- A rule that is true when A and B are **not independent**: 

$$ P(A|B) = \frac{P(A \mbox{ and } B)}{P(B)}$$
    
    - From that we can derive: 

$$P(A \mbox{ and } B) = P(A) P(B|A) = P(B) P(A|B)$$

- A rule that is true, when A and B are **independent**: $$P(A \mbox{ and } B) = P(A) P(B)$$. 


A and B are **independent** if the fact that A occurred, does not change the probability of B and viceversa. Trials corresponding to tossing a coin are independent. 

**Exercises**: 
+ If I have two children, what is the probability to have 2 girls?
+ If I have two children and we know that at least one of them is a girl, what is the probability that they are two girls?
+ If I have two children and we know that the older one is a girl, what is the probability that they are two girls?

In [ ]:
## Your solutions

### More probability rules

We say that two events are **mutually exclusive** if:

$$ P(A | B) = P(B | A) = 0 $$

In this case it is easy to show that:

$$ P(A \mbox{ or } B) = P(A) + P(B)$$

If A and B are not mutually exclusive:

$$ P(A \mbox{ or } B) = P(A) + P(B) - P(A \mbox{ and } B) $$

**Exercises**: Counting is the most basic skill to solve probability problems.

+ Q: For example, if I flip two coins, the chance of getting at least one tail is: 1/2 + 1/2?!
+ Q: If I roll two dices and the total is 8, what is the probability that one of the dice is 6?
+ Q: If I roll 100 dice, what is the probability of getting all sixes? 
+ Q: What is the probability of getting no sixes?
+ Q: What is the probability of getting at least one six?


In [ ]:
#Your solution here

## 3.2 Binomial distribution

More generally, the probability distribution that represents the probability of getting k times a success with probability p in n trials is:

$$ PMF(k) = {n \choose k} p^k (1-p)^{(n-k)}$$

where ${n \choose k} = \frac{n!}{k!(n-k)!}$. This is called **binomial distribution**.

What is the probability of having 2 successes in 5 trials?

In [ ]:
import scipy.special as sc
n = 5
k = 2
print(sc.comb(n, k, exact=True)) # Chances of 2 successes in 5 trials
p=sc.comb(n, k, exact=True)*1/2.0**2*(1-1/2.0)**(5-2)
print(p)

What is the probability of having 5 heads in 9 trials?


In [ ]:
# chances of 5 heads in 9 tosses

a = sc.comb(9, 5, exact=True)
print('The combinations of 9 on 5 are: ', a)

p = 0.5
print('Prob: ', a * p**5 * (1-p)**4)

**Exercise:** What is the probability of having 6 sixes in 9 trials?

​

## 3.3 Monte Carlo Experiments

​

**Monte Carlo experiments** are a broad class of computational algorithms that rely on *repeated random sampling to obtain numerical results*. Typically, one runs simulations many times over and over in order to obtain the distribution of an unknown probabilistic entity. (*Source: Wikipedia*)

​

**Trivial case**: What are the chances of getting a six in one trial?

​

In [ ]:
import random
import sys

N = 1000000 # perform N experiments
M = 0 # number of times, we got 6

for i in range(N):
    outcome = random.randint(1, 6)
    if outcome == 6:
        M += 1

Prob=M/float(N)
print('I got six %d times out of %d' % (M, N), '; Prob = ', 
      Prob, 'Note that: 1/6=', 1/6.0)


What are the chances of getting a six in two trials?

In [ ]:
#  chances of (exactly) 1 six in 2 trials

a = sc.comb(2, 1, exact=True)
p = 1/6.0

print('Prob: ', a * p * (1-p))

In [ ]:
N = 10000 # perform N experiments
M = 0 # no of times we get one 6

for i in range(N):
    outcome1 = random.randint(1, 6)
    outcome2 = random.randint(1, 6)
    if (outcome1 == 6 and outcome2 !=6) or (outcome1 != 6 and outcome2 == 6):
        M += 1

print('I got one six %d times out of %d' % (M, N), 
      '; Prob = ', float(M)/N)

**Exercise**: You throw two dice, one black and one red. What is the probability that the number of eyes on the black die is larger than the number of eyes on the red die?

In [ ]:
N = 10000 # perform N experiments
M = 0 # no of times we get one 6

for i in range(N):
    outcome1 = random.randint(1, 6)
    outcome2 = random.randint(1, 6)
    if (outcome1 > outcome2):
        M += 1
        
print('I got one six %d times out of %d' % (M, N), 
      '; Prob = ', float(M)/N)

**A more interesting case:** If I roll a dice 100 times, what is the chance of getting at least 6 sixes in a row?

In [ ]:
N = 10000 # perform N experiments
M = 0 # no of times we get one 6
T = 100
success='666666'

for i in range(N):
    outcome=''
    for i in range(T):

        outcome=outcome+str(random.randint(1,6))

    if (success in outcome):
            M += 1

print('I got 6 sixes in a row %d times out of %d' % (M, N), 

      '; Prob = ', float(M)/N)

**Exercise**: What is the probability that Messi scores at least 1 goal in a row of 10 matches during a season? (Let's suppose that each match is an independent trial).

**Data**: Messi scores 0.83 goals per match (323 goals in 387 matches) and CR4 scores 0.62 (329 goals in 527 matches) goals per match.

There are 42 matches in a season. 

In [ ]:
#your solution

<small>(Source: https://es.answers.yahoo.com/question/index?qid=20130928103148AAFQHsC)</small>

## 3.4 Continous distributions

So far, we have built **empirical distributions** (which represent the distributions of values in a sample), based on observations, but many real problems are well approximated by fitting **continous cumulative distributions functions (CDF)**. 

They are called in this way because the distribution is described by an analytic continuous function.

### 3.4.1 The exponential distribution

The CDF of the exponential distribution is:

$$ CDF(x) = 1 -  \exp^{- \lambda x}$$ 

And its PDF is:

$$ PDF(x) = \lambda \exp^{- \lambda x}$$

The parameter $\lambda$ determines the shape of the distribution, the mean of the distribution is $1/\lambda$ and its variance is $1/\lambda^2$. The median is $ln(2)/\lambda$.

In real applications, exponential distributions appear when we have a series of events and estimate the events times, called interarrival times. 

**When the events are equally likely to occur at any time, the interarrival times distribution used to get exponential distribution.**

As an example, the following figure shows the CDF of the interarrival times of birth in an Australian hospital. 44 births were registered in 24 hours, so the rate is  𝜆=0.0306  births/minute. The mean of the exponential distribution is  1/𝜆 , so the mean time between births is 32.7 minutes.

<center><img src='images/interarrivaltimes.png'></center>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

l = 2
x=np.arange(0,2.5,0.1)
y= 1 - np.exp(-l*x)

plt.plot(x,y,'-')
plt.title('Exponential CDF: $\lambda$ =%.2f' % l ,fontsize=15)
plt.xlabel('x',fontsize=15)
plt.ylabel('CDF',fontsize=15)
plt.show()

In [ ]:
from __future__ import division
import numpy as np
import scipy.stats as stats

l = 2
x=np.arange(0,2.5,0.1)
y= l * np.exp(-l*x)
plt.plot(x,y,'-')
plt.title('Exponential PDF: $\lambda$ =%.2f' % l, fontsize=15)
plt.xlabel('x', fontsize=15)
plt.ylabel('PDF', fontsize=15)
plt.show()

### Occurrence of Events

The exponential distribution occurs naturally when describing the lengths of the inter-arrival times in a homogeneous Poisson process.

In real-world scenarios, the assumption of a constant rate (or probability per unit time) is rarely satisfied. For example, the rate of incoming phone calls differs according to the time of day. But if we focus on a time interval during which the rate is roughly constant, such as from 2 to 4 p.m. during work days, the exponential distribution can be used as a good approximate model for the time until the next phone call arrives. Similar caveats apply to the following examples which yield approximately exponentially distributed variables:

- The time until a radioactive particle decays, or the time between clicks of a Geiger counter.
- The time it takes before your next telephone call
- The time until default (on payment to company debt holders) in reduced form credit risk modeling

The random variable $X$ of the lifelengths of some batteries is associated with a probability density function of the form:

$$ PDF(x) = \frac{1}{4} \exp^{- \frac{x}{4}}$$ 

In [ ]:
l = 0.25
x=np.arange(0,25,0.1)
y= l * np.exp(-l*x)
plt.plot(x,y,'-')
plt.title('Exponential: $\lambda$ =%.2f' % l ,fontsize=15)
plt.xlabel('x',fontsize=15)
plt.ylabel('PDF',fontsize=15)
plt.show()

### Example 1

The number of days ahead travelers purchase their airline tickets can be modeled by an exponential distribution with the average amount of time equal to 15 days. 

- (a) Find the probability that a traveler will purchase a ticket fewer than 10 days in advance. 


Answer: 

Remember: The parameter  𝜆  determines the shape of the distribution, the mean of the distribution is  1/𝜆. The median is  𝑙𝑛(2)/𝜆.

First we need to calculate the rate parameter: 𝜆=1/15=0.066667. Then we can calculate 𝑃(𝑋<10)=1−𝑒(−𝜆*10)=0.4865.

- (b) How many days do half of all travelers wait?

Answer:  
We know that the Median of the exponential distribution is 𝑀𝑒𝑑𝑖𝑎𝑛=(ln2)/𝜆=10.397

### Example 2

On the average, a certain device lasts 10 years. Assume that the length of time is exponentially distributed.

- a) What is the probability that the device lasts more than 7 years?

Answer: For the beginning 𝜆=1/10=0.1
 𝑃(𝑋>7)=1−𝑃(𝑋≤7)=1–(1−𝑒−𝜆𝑥)=𝑒−𝜆𝑥=𝑒−(0.1)(7)=0.4966

- b) On the average, how long would five devices last if they are used one after another?

Answer: On the average, 1 device lasts ten years. Therefore, 5 devices, if they are used one right after the other would last, on the average, (5)(10) = 50 years.

- c) What is the probability that a device lasts between 9 and 11 years?

Answer: 𝑃(9<𝑋<11)=𝑃(𝑋<11)−𝑃(𝑋<9)=(1−𝑒−(0.1)(11))–(1−𝑒−(0.1)(9))=0.0737

### 3.4.2 The normal distribution

The **normal or Gaussian distribution** is the most used one because it describes a lot of phenomena and because it is amenable for analysis. 

Its CDF has no closed-form expression and its more common representation is the PDF:

$$ PDF(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \left(-\frac{(x-\mu)^2}{2 \sigma^2} \right)$$


In [ ]:
u=5 # mean
s=1 # standard deviation
x=np.arange(0,15,0.1)
y=(1/(np.sqrt(2*np.pi*s*s)))*np.exp(-(((x-u)**2)/(2*s*s)))
plt.plot(x,y,'-')
plt.title('Gaussian PDF: $\mu$=%.1f, $\sigma$=%.1f' % (u,s),fontsize=15)
plt.xlabel('x',fontsize=15)
plt.ylabel('Probability density',fontsize=15)
plt.show()

Examples:
    * Measures of size of living tissue (length, height, skin area, weight);
    * The length of inert appendages (hair, claws, nails, teeth) of biological specimens, in the direction of growth; presumably the thickness of tree bark also falls under this category;
    * Certain physiological measurements, such as blood pressure of adult humans.


There exist much more distributions as: 
- the Pareto distribution (describing e.g. the distribution of wealth, cities sizes, sand particles, forest fires and earthquakes,
- the lognormal distribution (describing the adult weights), etc.


### 3.5 Central Limit Theorem

The normal distribution is also important, because it is involved in the Central Limit Theorem:

> Take the mean of $n$ random samples from ANY arbitrary distribution with a $well$ $defined$ standard deviation $\sigma$ and mean $\mu$. As $n$ gets bigger the **distribution of the sample mean** will always converge to a Gaussian (normal) distribution with mean $\mu$ and standard deviation $\frac{\sigma}{\sqrt{n}}$.

Colloquially speaking, the theorem states the distribution of an average tends to be normal, even when the distribution from which the average is computed is decidedly non-normal. This explains the ubiquity of the Gaussian distribution in science and statistics. 

#### Example: Uniform Distribution

The uniform distribution is obviously non-normal.  Let's call it the $parent$ $distribution$.

To compute an average, two samples are drawn ($n=2$), at random, from the parent distribution and averaged. Then another sample of two is drawn and another value of the average computed.  This process is repeated, over and over, and averages of two are computed.  

Repeatedly taking more elements ($n = 3,4...$) from the parent distribution, and computing the averages, produces a normal probability density.

In [ ]:
fig, ax = plt.subplots(1, 4, sharey=True, squeeze=True, figsize=(11, 5))
x = np.linspace(0, 1, 100)
for i in range(4):
    f = np.mean(np.random.random((10000, i+1)), 1)
    m, s = np.mean(f), np.std(f, ddof=1)
    fn = (1/(s*np.sqrt(2*np.pi)))*np.exp(-(x-m)**2/(2*s**2))  # normal pdf            
    ax[i].hist(f, 40, density=True, color=[0, 0.2, .8, .6]) 
    ax[i].set_title('n=%d' %(i+1))
    ax[i].plot(x, fn, color=[1, 0, 0, .6], linewidth=10)
    
plt.suptitle('Demonstration of the central limit theorem for a uniform distribution', y=1.05)
plt.show()

**The Central Limit Theorem explains the importance of normal distributions in the real world. Many features and properties of the living beings depend on genetic and environmental factors which effect usually is additive.** The measured features are sum of manny small effects that not necessarily follow the normal distributions, but their sum does follow according to the Central Limit Theorem.

## 3.6 Kernel density estimates 

In some instances, we may not be interested in the parameters of a particular distribution of data, but just a **continous representation** of the data at hand. In this case, we can estimate the distribution non-parametrically (i.e. making no assumptions about the form of the underlying distribution) using kernel density estimation.

Several uses are defined:

- **Visualization** - to explore the data by visualizing them and decide whether an estimated PDF is an appropriate model for the distribution.

- **Interpolation** - if we have reasons to beleive that the distribution is smooth, we can apply the KDE to interpolate the density specially for values that were not sampled.

- **Simulation** - specially when the sample distribution is small, it would be convenient to smooth the sample distribution by KDE in order to simulated and explore more possible outcomes, rather than replicating the observed data.

In [ ]:
from scipy.stats.distributions import norm

# Some random data
y = np.random.random(15) * 10
x = np.linspace(0, 10, 100)
# Smoothing parameter
s = 0.4

# Calculate the kernels
kernels = np.transpose([norm.pdf(x, yi, s) for yi in y])

plt.plot(x, kernels, 'k:')
plt.plot(x, kernels.sum(1))
plt.plot(y, np.zeros(len(y)), 'ro', ms=10)

SciPy implements a Gaussian KDE that automatically chooses an appropriate bandwidth. Let's create a bi-modal distribution of data that is not easily summarized by a parametric distribution:

In [ ]:
# Create a bi-modal distribution with a mixture of Normals.
x1 = np.random.normal(0, 3, 100) # parameters: (loc=0.0, scale=1.0, size=None)
x2 = np.random.normal(8, 1, 50)

# Append by row
x = np.r_[x1, x2] # r_ Translates slice objects to concatenation along the first axis.


plt.hist(x, bins=8, density=True)

In [ ]:
from scipy.stats import kde

density = kde.gaussian_kde(x)
xgrid = np.linspace(x.min(), x.max(), 100)
plt.hist(x, bins=18, density=True)
plt.plot(xgrid, density(xgrid), 'r-')

<font color='blue'>

## 4 Estimation
* 4.1 Sample mean
* 4.2 Variance
* 4.3 Standard scores
* 4.4 Covariance
* 4.5 Pearson,s correlation
* 4.6 Spearman's rank correlation
<font\> 


Let's think of a sequence of values: 
[-0.441, 1.774, -0.101, -1.138, 2.975, -2.138].

Can you guess which is the distribution? For example what would be its mean?

Hint: assume that it is normal distribution.


**Definition:** *Estimation* is the process of inferring the parameters (e.g. mean) of a distribution from a statistic of samples drown from a population.

For example: What is the estimated mean $\hat{\mu}$ of the following normal data?

In [ ]:
x = np.random.normal(0.0, 1.0, 10000)
a = plt.hist(x,50,density='True')

We can use our definition of empirical mean:

In [ ]:
print('The empirical mean of the sample is ', x.mean())

Let us imagine that we were reported the following data, where probably one of the data is wrong:

In [ ]:
x=np.array([-0.441, 1.774, -0.101, -1.138, 2.975, -213.8])
print(x.mean())

Is the mean estimator good enough?

### 4.1 Sample mean

+ The process is called **estimation** and the statistic we used **estimator**.

+ The median is also an estimator (more robust to outliers). 

+ "Is median better than sample mean?" is a question with at least two different answers. We can use two different objectives to answer this question: the minimization of error or the maximization to get the right answer. 

+ If there are no outliers, we can use the **sample mean** to minimize **mean squared error** (where $m$ is the number of times you play the estimation game, not the size of the sample!):

$$ MSE = \frac{1}{m} \sum(\hat{\mu} - \mu)^2$$


In [ ]:
err = 0.0
mu=0.0
NTests=1000
var=1.0
NPoints=100000
for i in range(NTests):
    x = np.random.normal(mu, var, NPoints)
    err += (mu - x.mean())**2

print('MSE: ', err/float(NTests) )

### 4.2 Variance

We can also estimate the variance with:

$$ \hat{\sigma}^2 = \frac{1}{n} \sum_i (x_i - \mu)^2 $$

This estimator works for large samples, but it is biased for small samples. We can use this one:

$$ \hat{\sigma}^2_{n-1} = \frac{1}{n-1} \sum_i (x_i - \mu)^2 $$


### 4.3 Other concepts: Standard scores

$$ z_i = \frac{x_i - \mu}{\sigma}$$

This measure is dimensionless and its distribution has mean 0 and variance 1.

It inherits the "shape" of $X$: if it is normally distributed, so is $Z$. If $X$ is skewed, so is $Z$.

### 4.4 Covariance (optional)

Sometimes it would be of interest to measure the relationship between two variables. 

**Covariance** is a measure of the tendency of two variables to vary together. 

If we have two series $X$ and $Y$ with $X=\{x_i\}$ and $Y=\{y_i\}$, and they vary together, their deviations $x_i - \mu_X$ and $y_i - \mu_Y$ tend to have the same sign.

If we multiply them together, the product is positive, when the deviations have the same sign, and negative, when they have the opposite sign. So adding up the products gives a measure of the tendency to vary together.

Covariance is the mean of the products:

$$ Cov(X,Y) = \frac{1}{n} \sum (x_i - \mu_X)*(y_i - \mu_Y), $$

where $n$ is the length of the two series.

It is a measure that is difficult to interpret.

In [ ]:
def Cov(X, Y):
    def _get_dvis(V):
        return [v - np.mean(V) for v in V]
    dxis = _get_dvis(X)
    dyis = _get_dvis(Y)
    return np.sum([x * y for x, y in zip(dxis, dyis)])/len(X)

X = [5, -1, 3.3, 2.7, 12.2]
Y=[10,12,8,9,11]

print("Cov(X, X) = %.2f" % Cov(X, X))
print("Var(X) = %.2f" % np.var(X))

print("Cov(X, Y) = %.2f" % Cov(X, Y))


<center><img src="images/gasolineprice.gif"></center>

### 4.5 Pearson's Correlation

Shell we take into account the variance? An alternative is to divide the deviations by $\sigma$, which yields standard scores, and compute the product of standard scores:

$$ p_i = \frac{(x_i - \mu_X)}{\sigma_X} \frac{(y_i - \mu_Y)}{\sigma_Y} $$
 
The mean of these products is:

$$ \rho = \frac{1}{n} \sum p_i = \frac{1}{n} \sum  \frac{(x_i - \mu_X)}{\sigma_X} \frac{(y_i - \mu_Y)}{\sigma_Y}  $$

Or we can rewrite $\rho$ by factoring out $\sigma_X$ and $\sigma_Y$:

$$ \rho = \frac{Cov(X,Y)}{\sigma_X \sigma_Y}$$

 


In [ ]:
def Corr(X, Y):
    assert len(X) == len(Y)
    return Cov(X, Y) / np.prod([np.std(V) for V in [X, Y]])

print("Corr(X, X) = %.5f" % Corr(X, X))

Y=np.random.random(len(X))

print("Corr(X, Y) = %.5f" % Corr(X, Y))

<center><img src="images/pearson.png"></center>

When $\rho = 0$, we cannot say that there is no relationship between the variables!

Pearson's coefficient only measures **linear** correlations!

### 4.6 Spearman’s rank correlation

Pearson’s correlation works well if the relationship between variables is linear and if the variables are roughly normal. But it is not robust in the presence of **outliers**.

Spearman’s rank correlation is an alternative that mitigates the effect of outliers and skewed distributions. To compute Spearman’s correlation, we have to compute the rank of each value, which is its index in the sorted sample. 

For example, in the sample {7, 1, 2, 5} the rank of the value 5 is 3, because it appears third if we sort the elements. 

Then, we compute the Pearson’s correlation, **but for the ranks**.

In [ ]:
def list2rank(l):
    #l is a list of numbers
    # returns a list of 1-based index; mean when multiple instances
    return [np.mean([i+1 for i, sorted_el in enumerate(sorted(l)) if sorted_el == el]) for el in l]

l = [7, 1, 2, 5]
print("ranks: ", list2rank(l))

def spearmanRank(X, Y):
    # X and Y are same-length lists
    return Corr(list2rank(X), list2rank(Y))

X = [1, 2, 3, 4, 100]
Y = [5, -100, 7, 10, 9]
plt.plot(X,'ro')
plt.plot(Y,'go')

print("Pearson rank coefficient: %.2f" % Corr(X, Y))
print("Spearman rank coefficient: %.2f" % spearmanRank(X, Y))


**Exercise:** Obtain for the Anscombe's quartet, the different estimators (mean, variance, covariance for each pair, Pearson's correlation and Spearman's rank correlation.

(Source: http://en.wikipedia.org/wiki/Anscombe's_quartet):

<center><img src="images/Anscombe's_quartet.png"></center>

In [ ]:
#Your solution here.


**Exercice:** Show if there is a correlation between the data of the babies....

In [ ]:
#Your solution here.

**Solution of the Messi's exercise:**

In [ ]:
N = 10000 # perform N experiments
M = 0 # no of times we get one 6
T = 42
Messi_score=0.83
Cristiano_score=0.62
success='1111111111'

for i in range(N):
    outcome=''
    for i in range(T):
        res=random.random()
        if res<Cristiano_score:
            outcome=outcome+'1'
        else: outcome=outcome+'0'

    if (success in outcome):
            M += 1

print('Cristiano got scores at least 1 goal in a row of 10 matches during a season %d times out of %d' % (M, N), 
      '; Prob = ', float(M)/N)

### Main reference

*Think Stats: Probability and Statistics for Programmers*, by Allen B. Downey, published by O'Reilly Media.

http://www.greenteapress.com/thinkstats/

